<a href="https://colab.research.google.com/github/MaFiN1337/crime-stats-ua/blob/main/parse-and-generate_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# generate_charts.py


In [ ]:
from google.colab import files
import shutil

shutil.make_archive('charts', 'zip', 'charts')

files.download('charts.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import plotly.express as px
import os

DATA_FILE = '/content/drive/MyDrive/DataSet_DaniTaSusp.xlsx'

OUTPUT_DIR = 'charts'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

try:
    df = pd.read_excel(DATA_FILE)
except FileNotFoundError:
    print(f"ПОМИЛКА: Файл не знайдено за шляхом {DATA_FILE}")
    print("Будь ласка, перевірте шлях і завантажте файл.")
    raise SystemExit

df['Date'] = pd.to_datetime(
    df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-01',
    errors='coerce'
)
df = df.dropna(subset=['Date'])
df = df.sort_values(by='Date')

articles_to_plot = df['Article'].unique()
years_to_plot = df['Year'].unique()

metrics_to_plot = [
    'Oblikovano_total',
    'Pidozra',
    'Zupineno',
    'Napravleno_do_sudu',
    'Provadjenya_zakryto',
    'Ne_priynyato_rishennya'
]

print(f"Знайдено статті: {articles_to_plot}")
print(f"Знайдено роки: {years_to_plot}")
print(f"Знайдено показники: {metrics_to_plot}")

total_charts = 0

print("Генеруємо річні (помісячні) графіки...")
for metric in metrics_to_plot:
    for article in articles_to_plot:
        for year in years_to_plot:

            data_subset = df[
                (df['Article'] == article) &
                (df['Year'] == year)
            ]

            if data_subset.empty:
                continue

            fig = px.bar(
                data_subset,
                x='Month',
                y=metric,
                title=f'Показник "{metric}" (помісячно)<br>Стаття {article}, {year} рік',
                labels={'Month': 'Місяць', metric: 'Кількість'}
            )
            fig.update_layout(xaxis=dict(tickmode='linear', dtick=1))

            filename = f"monthly_{article}_{year}_{metric}.html"
            save_path = os.path.join(OUTPUT_DIR, filename)

            fig.write_html(save_path)
            total_charts += 1

print(f"Згенеровано {total_charts} річних графіків.")
print("Генеруємо загальні (часові) графіки...")
for metric in metrics_to_plot:
    for article in articles_to_plot:

        data_subset = df[df['Article'] == article]

        if data_subset.empty:
            continue

        fig = px.line(
            data_subset,
            x='Date',
            y=metric,
            title=f'Загальна динаміка "{metric}"<br>Стаття {article} (2014-2024)',
            labels={'Date': 'Дата', metric: 'Кількість'},
            markers=True
        )

        filename = f"timeseries_{article}_{metric}.html"
        save_path = os.path.join(OUTPUT_DIR, filename)

        fig.write_html(save_path)
        total_charts += 1

print(f"УСЬОГО згенеровано {total_charts} HTML файлів у папці '{OUTPUT_DIR}'.")

Знайдено статті: [111 121 185 187 190]
Знайдено роки: [2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024]
Знайдено показники: ['Oblikovano_total', 'Pidozra', 'Zupineno', 'Napravleno_do_sudu', 'Provadjenya_zakryto', 'Ne_priynyato_rishennya']
Генеруємо річні (помісячні) графіки...
Згенеровано 330 річних графіків.
Генеруємо загальні (часові) графіки...
УСЬОГО згенеровано 360 HTML файлів у папці 'charts'.


## Отримання даних (2021+)

In [6]:
import pandas as pd
import re
from pathlib import Path

from glob import glob
files = glob('/content/*.xlsx')

COL_ARTICLE_DESC = 1
COL_OBLIKOVANO = 8
COL_PIDOZRA = 9
COLS_ZUPYNENO = [10, 11, 12, 13]
COLS_NAPRAVLENO = [14]
COL_ZAKRYTO = 18
COL_NE_PRYINYATO = 20

target_articles = ['111', '190', '121', '185', '187']

def extract_article_number(text):
    if pd.isna(text):
        return None
    match = re.search(r'ст\. ?(\d+)', str(text))
    if match:
        return match.group(1)
    return None

def process_file(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name='1', header=None, skiprows=2)
    except FileNotFoundError:
        print(f"❌ Файл не знайдено: {file_path}")
        return pd.DataFrame()
    except ValueError:
        print(f"❌ У файлі {file_path} немає аркуша '1'")
        return pd.DataFrame()

    df['Article_Number'] = df[COL_ARTICLE_DESC].apply(extract_article_number)
    df_filtered = df[df['Article_Number'].isin(target_articles)].copy()

    numeric_cols = [COL_OBLIKOVANO, COL_PIDOZRA, COL_ZAKRYTO, COL_NE_PRYINYATO] + COLS_ZUPYNENO + COLS_NAPRAVLENO
    for col in numeric_cols:
        df_filtered[col] = (
            df_filtered[col]
            .astype(str)
            .str.replace(',', '', regex=False)
        )
        df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce').fillna(0)

    df_filtered['Zupineno'] = df_filtered[COLS_ZUPYNENO].sum(axis=1)
    df_filtered['Napravleno_do_sudu'] = df_filtered[COLS_NAPRAVLENO].sum(axis=1)

    result_table = df_filtered.rename(columns={
        COL_ARTICLE_DESC: 'Стаття_Назва',
        COL_OBLIKOVANO: 'Oblikovano_total',
        COL_PIDOZRA: 'Pidozra',
        COL_ZAKRYTO: 'Provadjenya_zakryto',
        COL_NE_PRYINYATO: 'Ne_priynyato_rishennya'
    })[[
        'Article_Number',
        'Стаття_Назва',
        'Oblikovano_total',
        'Pidozra',
        'Zupineno',
        'Napravleno_do_sudu',
        'Provadjenya_zakryto',
        'Ne_priynyato_rishennya'
    ]]

    result_table['Стаття_Назва'] = result_table['Стаття_Назва'].astype(str).str.strip()
    result_table['Файл'] = Path(file_path).stem

    return result_table


# === Обробка всіх файлів ===
all_results = []
for file_path in files:
    print(f"🔹 Обробка файлу: {file_path}")
    result = process_file(file_path)
    if not result.empty:
        all_results.append(result)

if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
    output_file = '/content/combined_results.xlsx'
    final_df.to_excel(output_file, index=False)
    print(f"\n✅ Усі результати збережено у файл: {output_file}")
else:
    print("⚠️ Не вдалося знайти даних у жодному файлі.")



🔹 Обробка файлу: /content/forma_1_cherven_2019.xlsx

✅ Усі результати збережено у файл: /content/combined_results.xlsx
